In [45]:
import json
import requests

In [46]:
url = 'https://app.nanonets.com/api/v2/OCR/Model/08fe2528-e1e6-49c9-aff1-7aae3d13cba5/LabelFile/?async=false'

In [47]:
data = {'file': open('Receipt Photos copy/IMG_1789.JPG', 'rb')}

In [48]:
response = requests.post(url, auth=requests.auth.HTTPBasicAuth('Insert API Key', ''), files=data)
json_data = json.loads(response.text)
predictions = json_data.get("result", [])[0].get("prediction", [])
# print(response.text)

{"message":"Success","result":[{"message":"Success","input":"IMG_1789.JPG","prediction":[{"id":"36f6c193-7f87-4ba3-9a69-0e60e0f39878","label":"currency","xmin":0,"ymin":0,"xmax":0,"ymax":0,"score":1,"ocr_text":"USD","type":"field","status":"correctly_predicted","page_no":0,"label_id":"3fd0607f-e1de-49f7-ac9d-020013708650"},{"id":"7eec933e-1038-4298-9066-deb57c252e6b","label":"Merchant_Name","xmin":1092,"ymin":541,"xmax":1802,"ymax":592,"score":0.8997085,"ocr_text":"DIN TAI FUNG","type":"field","status":"correctly_predicted","page_no":0,"label_id":"e9449149-3cf2-4fe9-8c21-8f6a1b3088ba"},{"id":"32a9b219-9af9-4713-bbe1-caf86ef1ec7b","label":"Total_Amount","xmin":1790,"ymin":2199,"xmax":1975,"ymax":2257,"score":0.97512156,"ocr_text":"181.06","type":"field","status":"correctly_predicted","page_no":0,"label_id":"7d1b334c-8be1-4986-a438-0582c2954bc0"},{"id":"cb0af267-aa86-47da-baaa-cb331d99c1fc","label":"Tax_Amount","xmin":1814,"ymin":2053,"xmax":1974,"ymax":2109,"score":0.98706543,"ocr_text"

In [49]:
# Variables to store extracted data
merchant_name = ""
total_amount = ""
tax_amount = ""
date = ""
items = []

# Helper dictionary to organize items
item_details = {}

In [50]:
# Iterate through each prediction
for prediction in predictions:
    if prediction['label'] == 'Merchant_Name':
        merchant_name = prediction['ocr_text']
    elif prediction['label'] == 'Total_Amount':
        total_amount = prediction['ocr_text']
    elif prediction['label'] == 'Tax_Amount':
        tax_amount = prediction['ocr_text']
    elif prediction['label'] == 'Date':
        date = prediction['ocr_text']
    elif prediction['label'] == 'table':
        for cell in prediction['cells']:
            row = cell['row']
            label = cell['label']
            text = cell['text']
            if row not in item_details:
                item_details[row] = {'Quantity': '', 'Description': '', 'Line_Amount': ''}
            if label in ['Quantity', 'Description', 'Line_Amount']:
                item_details[row][label] = text

# Convert item details to a list of structured dictionaries
item_list = []
for item in item_details.values():
    item_list.append({'Name': item['Description'], 'Price': item['Line_Amount']})

In [51]:
final_output = {
    'Merchant Name': merchant_name,
    'Total Amount': total_amount,
    'Tax Amount': tax_amount,
    'Date': date,
    'Items': item_list
}

In [59]:
final_output


{'Merchant Name': 'DIN TAI FUNG',
 'Total Amount': '181.06',
 'Tax Amount': '14.26',
 'Date': '2023-07-18',
 'Items': [{'Name': 'Seaweed & Beancurd Salad', 'Price': '$ 7.50'},
  {'Name': 'Sweet & Sour Pork Baby Back Ribs', 'Price': '$ 14.50'},
  {'Name': 'Hot & Sour Soup', 'Price': '$ 12.50'},
  {'Name': 'Pork Xiao Long Bao', 'Price': '$ 15.50'},
  {'Name': 'Sticky Rice & Pork Shao Mai', 'Price': '$ 10.50'},
  {'Name': 'Taiwanese Cabbage w / Garlic', 'Price': '$ 14.00'},
  {'Name': 'Vegan Dumplings', 'Price': '$ 15.50'},
  {'Name': 'Shrimp & Pork Spicy Wontons', 'Price': '$ 15.00'},
  {'Name': 'Pork Chop Fried Rice', 'Price': '$ 18.00'},
  {'Name': 'Chicken Shanghai Rice Cakes', 'Price': '$ 16.00'}]}

In [58]:
final_output['Items'][0]['Name']

'Seaweed & Beancurd Salad'